In [1]:
# 共通で利用するライブラリ
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings

# warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, precision=3)
pd.options.display.float_format = "{:.3f}".format
pd.set_option("display.max_columns", None)
plt.style.use("seaborn")
plt.rcParams["font.size"] = 14
plt.rcParams["font.family"] = "IPAexGothic"
%matplotlib inline

# 追加するライブラリ
from glob import glob
import os
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Border, Side, Font
from openpyxl.chart import Reference, BarChart, PieChart, LineChart, ScatterChart, Series


## Ch.04 レポーティングする仕組みを構築する

データ一覧

- m_area.csv
  - 地域マスタ
  - 都道府県情報等
- m_store.csv
  - 店舗マスタ
  - 店舗名等
- tbl_order_202004.csv
  - 注文データ
  - 4月分
- tbl_order_202005.csv
  - 注文データ
  - 5月分
- tbl_order_202006.csv
  - 注文データ
  - 6月分

### Knock31: 特定店舗の売上をExcelにして出力する

In [2]:
# データの読み込み

# データの読み込み
data_dir = "../support/本章/4章"
m_store = pd.read_csv(data_dir + "/m_store.csv")
m_area = pd.read_csv(data_dir + "/m_area.csv")
tbl_order_file = os.path.join(data_dir, "tbl_order_*.csv")
tbl_order_files = glob(tbl_order_file)

order_all = pd.DataFrame()
for file in tbl_order_files:
    order_tmp = pd.read_csv(file)
    print(f"{file}: {order_tmp.shape}")
    order_all = pd.concat([order_all, order_tmp], axis=0, ignore_index=True)

# 保守用店舗データの削除
order_all = order_all.loc[order_all["store_id"] != 999]

# マスタデータを結合
order_all = pd.merge(order_all, m_store, on="store_id", how="left")
order_all = pd.merge(order_all, m_area, on="area_cd", how="left")

# マスタデータにないコードに対応した文字列の設定
# ["takeout_name"]の追加
order_all.loc[order_all["takeout_flag"] == 0, "takeout_name"] = "デリバリー"
order_all.loc[order_all["takeout_flag"] == 1, "takeout_name"] = "お持ち帰り"
# ["status_name"]の追加
order_all.loc[order_all["status"] == 0, "status_name"] = "受付"
order_all.loc[order_all["status"] == 1, "status_name"] = "お支払済"
order_all.loc[order_all["status"] == 2, "status_name"] = "お渡し済"
order_all.loc[order_all["status"] == 9, "status_name"] = "キャンセル"

# ["order_date"]の追加
order_all.loc[:, "order_date"] = pd.to_datetime(
    order_all["order_accept_date"]).dt.date

print(order_all.shape)
display(order_all)


../support/本章/4章/tbl_order_202006.csv: (233301, 10)
../support/本章/4章/tbl_order_202004.csv: (233260, 10)
../support/本章/4章/tbl_order_202005.csv: (241139, 10)
(703880, 17)


,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name,order_date
0,16514392,64,C44271355,49,56527968,2020-06-01 11:00:00,2020-06-01 11:55:00,0,2732,2,西東京店,TK,東京,東京,デリバリー,お渡し済,2020-06-01
1,28342394,12,C98307866,98,46924304,2020-06-01 11:00:00,2020-06-01 11:52:00,0,2064,2,西葛西店,TK,東京,東京,デリバリー,お渡し済,2020-06-01
2,29799587,174,C59484037,45,47054474,2020-06-01 11:00:00,2020-06-01 11:25:00,0,3900,2,新座店,SA,埼玉,埼玉,デリバリー,お渡し済,2020-06-01
3,96960753,23,C57278332,70,60848267,2020-06-01 11:00:00,2020-06-01 11:45:00,0,2750,2,江東店,TK,東京,東京,デリバリー,お渡し済,2020-06-01
4,52147209,7,C36890849,80,20633593,2020-06-01 11:00:00,2020-06-01 11:23:00,0,2328,2,東尾久店,TK,東京,東京,デリバリー,お渡し済,2020-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703875,47039360,154,C39765066,90,27121351,2020-05-31 21:58:58,2020-05-31 22:42:58,1,2363,1,入間店,SA,埼玉,埼玉,お持ち帰り,お支払済,2020-05-31
703876,61054188,94,C19465010,41,61513298,2020-05-31 21:58:58,2020-05-31 22:40:58,0,2112,2,鎌倉店,KN,神奈川,神奈川,デリバリー,お渡し済,2020-05-31
703877,45288914,11,C23883838,40,65359139,2020-05-31 21:58:58,2020-05-31 22:38:58,0,3838,2,江戸川店,TK,東京,東京,デリバリー,お渡し済,2020-05-31
703878,41608372,149,C48350551,43,42138977,2020-05-31 21:58:58,2020-05-31 22:52:58,0,1899,2,松戸店,CH,千葉,千葉,デリバリー,お渡し済,2020-05-31


In [3]:
# Excelの書き込みテスト

wb = openpyxl.Workbook()
ws = wb["Sheet"]
ws.cell(1, 1).value = "書き込みのテストです。"
wb.save("test.xlsx")
wb.close()

In [4]:
# Excelの読み込みテスト

wb = openpyxl.load_workbook("test.xlsx", read_only=True)
ws = wb["Sheet"]
print(ws.cell(1, 1).value)
wb.close()

書き込みのテストです。


In [5]:
# テストデータの準備

store_id = 1
store_df = order_all.loc[order_all["store_id"] == store_id].copy()
store_name = store_df["store_name"].unique()[0]
store_sales_total = store_df.loc[store_df["status"].isin([1, 2])]["total_amount"].sum()
store_sales_takeout = store_df.loc[store_df["status"] == 1]["total_amount"].sum()
store_sales_delivery = store_df.loc[store_df["status"] == 2]["total_amount"].sum()

print(f"売上額の確認 {store_sales_total} = {store_sales_takeout} + {store_sales_delivery}")
print(store_sales_total == (store_sales_takeout + store_sales_delivery))

output_df = store_df[[
    "order_accept_date", "customer_id", "total_amount",
    "takeout_name", "status_name"
]]

print(output_df.shape)
display(output_df)

売上額の確認 9004535 = 2255499 + 6749036
True
(3553, 5)


,order_accept_date,customer_id,total_amount,takeout_name,status_name
58,2020-06-01 11:05:05,C13019060,3530,デリバリー,お渡し済
222,2020-06-01 11:16:16,C42292941,3617,デリバリー,お渡し済
309,2020-06-01 11:24:24,C04466742,2877,お持ち帰り,お支払済
641,2020-06-01 11:52:52,C05244872,2987,お持ち帰り,お支払済
1118,2020-06-01 12:31:31,C27811735,1882,お持ち帰り,お支払済
...,...,...,...,...,...
702927,2020-05-31 20:35:35,C82128714,3300,デリバリー,お渡し済
703101,2020-05-31 20:50:50,C44652586,4144,お持ち帰り,お支払済
703113,2020-05-31 20:51:51,C69752586,2471,デリバリー,お渡し済
703212,2020-05-31 21:01:01,C56303016,4692,デリバリー,キャンセル


In [6]:
# データをExcelに出力

store_title = f"{store_id}_{store_name}"

wb = openpyxl.Workbook()
ws = wb.active
ws.title = store_title

ws.cell(1, 1).value = f"{store_title} 売上データ"

# OpenPyXLのユーティリティdataframe_to_rowsを利用する
rows = dataframe_to_rows(output_df, index=False, header=True)

# 表の貼付け位置の指定
row_start = 3
col_start = 2

for row_no, row in enumerate(rows, row_start):
    for col_no, value in enumerate(row, col_start):
        ws.cell(row_no, col_no).value = value

filename = f"{store_title}.xlsx"
wb.save(filename)
wb.close()

### Knock32: Excelの表を整えて出力する

In [7]:
# 罫線の設定と描画

openpyxl.load_workbook(filename)
ws =wb[store_title]

side = Side(style="thin", color="008080")
border = Border(top=side, bottom=side, left=side, right=side)

# データの表の部分に罫線を設定する
for row in ws:
    for cell in row:
        if ws[cell.coordinate].value:
            ws[cell.coordinate].border = border

In [8]:
ws.cell(1, 1).font = Font(bold=True, color="008080")

cell = ws.cell(3, 2)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "注文受注日時"
cell.font = Font(bold=True, color="FFFFFF")

cell = ws.cell(3, 3)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "顧客ID"
cell.font = Font(bold=True, color="FFFFFF")

cell = ws.cell(3, 4)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "購入総額"
cell.font = Font(bold=True, color="FFFFFF")

cell = ws.cell(3, 5)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "注文タイプ"
cell.font = Font(bold=True, color="FFFFFF")

cell = ws.cell(3, 6)
cell.fill = PatternFill(patternType="solid", fgColor="008080")
cell.value = "注文状態"
cell.font = Font(bold=True, color="FFFFFF")

ws.column_dimensions["A"].width = 20
ws.column_dimensions["B"].width = 20
ws.column_dimensions["C"].width = 12
ws.column_dimensions["D"].width = 12
ws.column_dimensions["E"].width = 12
ws.column_dimensions["F"].width = 12

# ファイルに保存
wb.save(filename)
wb.close()

### Knock33: 売上以外のデータも出力する

In [9]:
# 「配達完了までの時間」を計算する

def calc_delta(t):
    """配達完了までの時間を計算する"""
    t1, t2 = t
    delta = t2 - t1
    # 経過時間を分単位で返す
    return delta.total_seconds()/60

store_df.loc[:, "order_accept_datetime"] = pd.to_datetime(store_df["order_accept_date"])
store_df.loc[:, "delivered_datetime"] = pd.to_datetime(store_df["delivered_date"])
store_df.loc[:, "delta"] = store_df[["order_accept_datetime", "delivered_datetime"]].apply(calc_delta, axis=1)

delivery_time = store_df.groupby(["store_id"])["delta"].describe()
display(delivery_time)

,count,mean,std,min,25%,50%,75%,max
store_id,,,,,,,,
1,3553.000,34.477,14.514,10.000,22.000,34.000,47.000,59.000


In [10]:
# 計算した「配達完了までの時間」をExcelに出力

openpyxl.load_workbook(filename)
ws = wb[store_title]

cell = ws.cell(1, 7)
cell.value = f"配達完了までの時間"
cell.font = Font(bold=True, color="008080")

rows = dataframe_to_rows(delivery_time, index=False, header=True)

# 表の貼り付け位置の設定
row_start = 3
col_start = 8

for row_no, row in enumerate(rows, row_start):
    for col_no, value in enumerate(row, col_start):
        cell = ws.cell(row_no, col_no)
        cell.value = value
        cell.border = border
        if row_no == row_start:
            cell.fill = PatternFill(patternType="solid", fgColor="008080")
            cell.font = Font(bold=True, color="FFFFFF")

filename = f"{store_title}.xlsx"
wb.save(filename)
wb.close()


### Knock34: 問題のある箇所を赤字で出力する

In [11]:
# キャンセルデータを赤文字にして出力

openpyxl.load_workbook(filename)
ws = wb[store_title]

rows = dataframe_to_rows(output_df, index=False, header=True)

# 表の貼付け位置の設定
row_start = 3
col_start = 2

for row_no, row in enumerate(rows, row_start):
    if row_no == row_start:
        continue
    for col_no, value in enumerate(row, col_start):
        ws.cell(row_no, col_no).value = value
        if value == "キャンセル":
            ws.cell(row_no, col_no).font = Font(bold=False, color="FF0000")

filename = f"{store_title}.xlsx"
wb.save(filename)
wb.close()


### Knock35: Excelのセル関数で日ごとの集計する

In [12]:
# Excelのセル関数

openpyxl.load_workbook(filename)
ws = wb[store_title]

cell = ws.cell(7, 7)
cell.value = "集計"
cell.font = Font(bold=True, color="008080")

cell = ws.cell(8, 8)
cell.value = "データ総額"
cell.font = Font(bold=True, color="008080")

cell = ws.cell(8, 10)
cell.value = f'=SUM(D4:D{ws.max_row})'

cell = ws.cell(9, 8)
cell.value = "  内 決済完了額"
cell.font = Font(bold=True)

cell = ws.cell(9, 10)
cell.value = f'=SUMIF(F4:F{ws.max_row},"<>キャンセル",D4:D{ws.max_row})'

cell = ws.cell(10, 8)
cell.value = "  内 キャンセル額"
cell.font = Font(bold=True)

cell = ws.cell(10, 10)
cell.value = f'=SUMIF(F4:F{ws.max_row},"=キャンセル",D4:D{ws.max_row})'

# print(ws.max_row, ws.max_column)  # (3556, 15)

filename = f"{store_title}.xlsx"
wb.save(filename)
wb.close()


### Knock36: 折れ線グラフにして出力する

In [13]:
# グラフ描画処理

openpyxl.load_workbook(filename)
ws = wb[store_title]

cell = ws.cell(7, 7)
cell.value = f'売上グラフ'
cell.font = Font(bold=True, color="008080")

# グラフ用の参照データを指定
# D列（購入総額）の4行目から20件を指定
refy = Reference(ws, min_col=4, min_row=4, max_col=4, max_row=23)

# グラフシリーズを生成
series = Series(refy, title="売上額")

# Chart
chart = LineChart()
chart.title = "折れ線グラフ"
chart.x_axis.title = "件数"
chart.y_axis.title = "売上額"
chart.height = 10
chart.width = 20
chart.series.append(series)

# 生成したChartオブジェクトをシートの指定位置に追加する
ws.add_chart(chart, "H12")

filename = f"{store_title}.xlsx"
wb.save(filename)
wb.close()


### Knock37: レポートに向けてデータを準備する

In [15]:
# キャンセル率ランキングと店舗ランクの調査関数

# キャンセル率ランキングデータを準備
cancel_df = pd.DataFrame()
cancel_cnt = order_all.loc[order_all["status"] == 9].groupby(["store_id"])["store_id"].count()
order_cnt = order_all.loc[order_all["status"].isin([1, 2, 9])].groupby(["store_id"])["store_id"].count()
cancel_rate = (cancel_cnt / order_cnt) * 100
cancel_df["cancel_rate"] = cancel_rate
cancel_df = pd.merge(cancel_df, m_store, on="store_id", how="left")
cancel_rank = cancel_df.sort_values("cancel_rate", ascending=True).reset_index()

def check_store_cancel_rank(trg_id):
    """指定した店舗の「キャンセル率」ランキングの順位を返す"""
    tmp = cancel_rank.loc[cancel_rank["store_id"] == trg_id].index + 1
    return tmp[0]

In [17]:
# 地域ごとの売上集計関数とランキング集計

def get_area_df(trg_id):
    """該当店舗が属する地域の注文情報を抽出する"""
    area_df = pd.DataFrame()
    area_df = order_all.loc[order_all["area_cd"] == store_df["area_cd"].unique()[0]]
    area_df = area_df.loc[area_df["status"].isin([1, 2])]
    return area_df

def get_area_rank_df(trg_id):
    """
    該当する地域の注文情報から店舗単位で金額合計を算出する
    金額の多い順に並べ替えてランキングを作成する
    """
    area_df = get_area_df(trg_id)
    area_rank = area_df.groupby(["store_id"])["total_amount"].sum().sort_values(ascending=False)
    area_rank = pd.merge(area_rank, m_store, on="store_id", how="left")
    return area_rank

def check_store_sales_rank(trg_id):
    """指定した店舗の「売上」ランキングの順位を返す"""
    area_rank = get_area_rank_df(trg_id)
    tmp = area_rank.loc[area_rank["store_id"] == trg_id].index + 1
    return tmp[0]

In [27]:
# 該当店舗の日次売上を集計

def make_store_daily(trg_id):
    """該当店舗の日次売上の集計"""
    tmp_store_df = order_all.loc[
        (order_all["store_id"] == trg_id) &
        (order_all["status"].isin([1, 2]))
    ]
    tmp = tmp_store_df[["order_accept_date", "total_amount"]].copy()
    tmp.loc[:, "order_accept_date"] = pd.to_datetime(tmp["order_accept_date"])
    tmp.set_index("order_accept_date", inplace=True)
    tmp = tmp.resample("D").sum().reset_index()
    return tmp


In [25]:
# 該当店舗の配達完了までの時間の集計処理

def get_area_delivery(trg_id):
    """
    「配達完了までの時間」の計算結果をデータフレームに格納する
    """
    area_delivery = pd.DataFrame()
    area_df = get_area_df(trg_id)
    area_delivery = area_df.loc[area_df["status"] == 2].copy()
    area_delivery.loc[:, "order_accept_datetime"] = pd.to_datetime(area_delivery["order_accept_date"])
    area_delivery.loc[:, "delivered_datetime"] = pd.to_datetime(area_delivery["delivered_date"])
    area_delivery.loc[:, "delta"] = area_delivery[["order_accept_datetime", "delivered_datetime"]].apply(calc_delta, axis=1)
    return area_delivery

def get_area_delivery_rank_df(trg_id):
    """
    店舗ごとの「配達完了までの時間」の平均を計算する
    時間の速い順に並べ替えてデータフレームに格納する
    """
    area_delivery = get_area_delivery(trg_id)
    area_delivery_rank = area_delivery.groupby(["store_id"])["delta"].mean().sort_values()
    area_delivery_rank = pd.merge(area_delivery_rank, m_store, on="store_id", how="left")
    return area_delivery_rank

def check_store_delivery_rank(trg_id):
    """指定した店舗の「配達完了までの時間」ランキングの順位を返す"""
    area_delivery_rank = get_area_delivery_rank_df(trg_id)
    tmp = area_delivery_rank.loc[area_delivery_rank["store_id"] == trg_id].index + 1
    return tmp[0]
    

### Knock38: データシートに必要なデータを出力する

In [21]:
# テスト用ファイルの削除

if os.path.exists("test.xlsx") : os.remove("test.xlsx")
if os.path.exists(filename) : os.remove(filename)

In [29]:
# データをExcelに出力する汎用関数

def data_sheet_output(trg_wb, sheetname: str, target_df: pd.DataFrame, index_flag: bool):
    """データをExcelに出力する"""
    ws = trg_wb.create_sheet(title=sheetname)
    
    rows = dataframe_to_rows(target_df, index=index_flag, header=True)

    # 表の貼り付け位置の設定
    row_start = 1
    col_start = 1
    for row_no, row in enumerate(rows, row_start):
        for col_no, value in enumerate(row, col_start):
            ws.cell(row_no, col_no).value = value

    # データシートは非表示にする
    ws.sheet_state = "hidden"

In [23]:
# data_sheet_output関数をコールする関数

def make_data_sheet(trg_id, trg_store_df: pd.DataFrame, target_folder: str):
    """
    data_sheet_output関数をコールする
    必要なデータをシートに出力する
    """
    # target_daily = make_store_daily(trg_id)
    store_name = trg_store_df["store_name"].unique()[0]

    # ファイルを作成する
    store_title = f"{trg_id}_{store_name}"

    wb = openpyxl.Workbook()

    # 「キャンセル率」ランキング
    data_sheet_output(wb, "Data_Cancel_Rank", cancel_rank, index_flag=False)
    
    # エリア「売上」ランキング
    data_sheet_output(wb, "Data_Area_Rank", get_area_rank_df(trg_id), index_flag=False)
    
    # エリア「配達時間完了までの時間」ランキング
    data_sheet_output(wb, "Data_Delivery_Rank", get_area_delivery_rank_df(trg_id), index_flag=False)

    # 該当店の日次売上データ
    data_sheet_output(wb, "Data_Target_Daily", make_store_daily(trg_id), index_flag=False)
    
    filename = os.path.join(target_folder, f"{store_title}.xlsx")
    wb.save(filename)
    wb.close()
    return filename

In [30]:
# make_data-sheet関数呼び出し
filename_store = make_data_sheet(store_id, store_df, "")

### Knock39: サマリーシートを作成する

In [33]:
# サマリーシートの作成関数

def make_summary_sheet(trg_id, store_name, trg_file):
    """"""
    target_cancel_rank = check_store_cancel_rank(trg_id)
    target_sales_rank = check_store_sales_rank(trg_id)
    target_delivery_rank = check_store_delivery_rank(trg_id)

    wb = openpyxl.load_workbook(trg_file)
    ws = wb.active
    ws.title = "サマリーレポート"

    cell = ws.cell(1, 1)
    cell.value = f"{store_name} サマリーリポート（4月〜6月）"
    cell.font = Font(bold=True, color="008080", size=20)

    # 売上ランキングの表示
    tmp_ws = wb["Data_Target_Daily"]
    cell = ws.cell(3, 2)
    cell.value = "店舗売上額"
    cell.font = Font(bold=True, color="008080", size=16)

    # セルの結合
    ws.merge_cells("E3:F3")

    cell = ws.cell(3, 5)
    cell.value = f"=SUM({tmp_ws.title}!B2:B{tmp_ws.max_row})"
    cell.font = Font(bold=True, color="0080FF", size=16)
    cell.number_format = "#,##0"

    cell = ws.cell(4, 2)
    cell.value = "店舗売上ランク"
    cell.font = Font(bold=True, color="008080", size=16)

    cell = ws.cell(4, 5)
    cell.value = f"{len(m_store)}店舗中 {target_sales_rank} 位"
    cell.font = Font(bold=True, color="0080FF", size=16)

    # グラフ用の参照データを指定
    refy = Reference(tmp_ws, min_col=2, min_row=2, max_col=2, max_row=tmp_ws.max_row)

    # グラフシリーズを生成
    series = Series(refy, title="売上額")

    # Chart
    chart = LineChart()
    chart.title = "期間売上額（日次）"
    chart.x_axis.title = "件数"
    chart.y_axis.title = "売上額"
    chart.height = 10
    chart.width = 15
    chart.series.append(series)

    # 生成したChartオブジェクトをシートの指定位置に追加
    ws.add_chart(chart, "B6")

    # 地域情報
    tmp_ws = wb["Data_Area_Rank"]

    cell = ws.cell(4, 10)
    cell.value = "地域店舗売上情報"
    cell.font = Font(bold=True, color="008080", size=16)

    cell = ws.cell(5, 11)
    cell.value = "最高額"

    cell = ws.cell(5, 12)
    cell.value = f"=MAX({tmp_ws.title}!B2:B{tmp_ws.max_row})"
    cell.number_format = "#,##0"

    cell = ws.cell(6, 11)
    cell.value = "最低額"

    cell = ws.cell(6, 12)
    cell.value = f"=MIN({tmp_ws.title}!B2:B{tmp_ws.max_row})"
    cell.number_format = "#,##0"

    cell = ws.cell(7, 11)
    cell.value = "地域平均"

    cell = ws.cell(7, 12)
    cell.value = f"=AVERAGE({tmp_ws.title}!B2:B{tmp_ws.max_row})"
    cell.number_format = "#,##0"

    # キャンセル率の表示
    cell = ws.cell(11, 10)
    cell.value = "キャンセル率ランク"
    cell.font = Font(bold=True, color="008080", size=16)

    cell = ws.cell(12, 11)
    cell.value = f"{len(m_store)}店舗中 {target_cancel_rank} 位"
    cell.font = Font(bold=True, color="0080FF", size=16)

    tmp_ws = wb["Data_Cancel_Rank"]

    cell = ws.cell(13, 11)
    cell.value = "地域平均"

    cell = ws.cell(13, 12)
    cell.value = f"=AVERAGE({tmp_ws.title}!C2:C{tmp_ws.max_row})"
    cell.number_format = "0.00"

    # 配達時間ランキングの表示
    cell = ws.cell(15, 10)
    cell.value = "配達時間ランク"
    cell.font = Font(bold=True, color="008080", size=16)

    cell = ws.cell(16, 11)
    cell.value = f"{len(m_store)}店舗中 {target_delivery_rank} 位"
    cell.font = Font(bold=True, color="0080FF", size=16)

    tmp_ws = wb["Data_Delivery_Rank"]

    cell = ws.cell(17, 11)
    cell.value = "地域平均"

    cell = ws.cell(17, 12)
    cell.value = f"=AVERAGE({tmp_ws.title}!B2:B{tmp_ws.max_row})"
    cell.number_format = "0.00"
    
    wb.save(trg_file)
    wb.close()


In [34]:
# サマリーシート作成関数の実行

make_summary_sheet(store_id, store_name, filename_store)

### Knock40: 店舗別にレポートをExcel出力する

In [36]:
# 店舗ループによる全店舗のレポート出力

os.makedirs("output", exist_ok=True)

count = 0
total = len(m_store)
for store in m_store["store_id"].to_list():
    if store != 999:
        store_df = order_all.loc[order_all["store_id"] == store]
        store_name = m_store.loc[m_store["store_id"] == store]["store_name"]

        count += 1
        print(f"{count}/{total}: {store_name}")

        tmp_filename = make_data_sheet(store, store_df, "output")
        make_summary_sheet(store, store_name.values[0], tmp_filename)

print("出力完了しました。")
# 50m

1/197: 0    昭島店
Name: store_name, dtype: object
2/197: 1    あきる野店
Name: store_name, dtype: object
3/197: 2    足立店
Name: store_name, dtype: object
4/197: 3    北千住店
Name: store_name, dtype: object
5/197: 4    綾瀬店
Name: store_name, dtype: object
6/197: 5    荒川店
Name: store_name, dtype: object
7/197: 6    東尾久店
Name: store_name, dtype: object
8/197: 7    板橋店
Name: store_name, dtype: object
9/197: 8    高島平店
Name: store_name, dtype: object
10/197: 9    稲城店
Name: store_name, dtype: object
11/197: 10    江戸川店
Name: store_name, dtype: object
12/197: 11    西葛西店
Name: store_name, dtype: object
13/197: 12    青梅店
Name: store_name, dtype: object
14/197: 13    大田店
Name: store_name, dtype: object
15/197: 14    大森店
Name: store_name, dtype: object
16/197: 15    蒲田店
Name: store_name, dtype: object
17/197: 16    葛飾店
Name: store_name, dtype: object
18/197: 17    亀有店
Name: store_name, dtype: object
19/197: 18    赤羽店
Name: store_name, dtype: object
20/197: 19    王子店
Name: store_name, dtype: object
21/197: 20  